In [7]:
import glob
import os
import png
import numpy as np
#import pydicom as dcm 
#from pydicom.pixel_data_handlers.util import apply_voi_lut
import PIL.ImageOps 
from PIL import Image
import cv2
import PIL
import os
import natsort
import shutil

* Excel파일과 연동하여 group을 분류하는

In [5]:
import openpyxl
from openpyxl import load_workbook
import pandas as pd
#엑셀을 불러옵니다.
patient_data = load_workbook('C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/Bone scan (1).xlsx')
#지정한 시트를 불러옵니다.
excel_data = patient_data['Sheet1']
#데이터프레임으로 변환합니다.
df = pd.DataFrame(excel_data.values)

#변환된 데이터 프레임을 출력합니다.
display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,ExamDate,Section,Modality,ExamCode,ExamName,PatID,Sex,Age,Birthday,ReportDate,Conclusion,Group,Result,Finding,Recommend,Radiologist,Status,None
1,2019-07-01 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),170293173,F,65,1956-12-05 00:00:00,2019-07-02 00:00:00,Breast cancer,1,1,1. No definite evidence of bone metastasis_x00...,None,이석현(181356)/,C,None
2,2019-07-01 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),910476443,F,74,1947-12-25 00:00:00,2019-07-02 00:00:00,RCC,2,1,1. No definite evidence of bone metastasis_x00...,None,이석현(181356)/,C,None
3,2019-07-02 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),990328273,F,52,1969-10-03 00:00:00,2019-07-03 00:00:00,Breast cancer,1,0,No definite evidence of bone metastasis\n,None,이석현(181356)/,C,None
4,2019-07-02 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),040309243,F,65,1957-02-15 00:00:00,2019-07-03 00:00:00,Breast cancer,1,1,1. No definite evidence of bone metastasis_x00...,None,이석현(181356)/,C,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6504,2022-06-30 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),170407993,F,83,1939-04-01 00:00:00,2022-07-01 00:00:00,Breast cancer,1,1,1. No definite evidence of bone metastasis_x00...,None,이석현(181356)/,C,None
6505,2022-06-30 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),220315273,M,67,1955-05-01 00:00:00,2022-07-01 00:00:00,Prostate cancer,1,1,1. No definite evidence of bone metastasis_x00...,None,이석현(181356)/,C,None
6506,2022-06-30 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),120168183,M,79,1942-10-14 00:00:00,2022-07-01 00:00:00,Bladder tumor,2,0,No definite evidence of bone metastasis\n,None,이석현(181356)/,C,None
6507,2022-06-30 00:00:00,RI,NM,HC191R,NM (Bone Whole Body Scan),220334833,F,38,1984-01-06 00:00:00,2022-07-01 00:00:00,Pain,3,0,No active bone lesion\n,None,이석현(181356)/,C,None


In [73]:
#위에서 보시면 column명을 0번째 행으로 설정한 후 0번째 행은 제거해줍니다. 
df.columns = df.iloc[0]
df = df.drop(0)
df['ExamDate'] = df['ExamDate'].astype(str) #datetime(시간형식)형식으로 되어있어서 str형식(일반문자)으로 변환해줍니다.

In [74]:
#파일명의 Study Date와 비교하기위해 동일한 형식으로 변환(제거)합니다.
df['ExamDate'] = df['ExamDate'].replace('-','',regex = True)
df['ExamDate'] = df['ExamDate'].replace('00:00:00','',regex = True)

# 참고하실 내용

여기보시면 제가 result에 9라고 된 환자들을 출력하고 있습니다.

Excel표에 result 부분이 N/A로 표시된 환자들을 임의로 9로 변환, no_final_diagnosis로 표시된 환자들을 8로 변환하였습니다.

위 환자들은 결과에서 제외됩니다.

In [75]:
####
df['Result'][df['Result'] == 9]

39      9
154     9
209     9
250     9
276     9
       ..
6160    9
6203    9
6212    9
6283    9
6327    9
Name: Result, Length: 123, dtype: object

In [60]:
done_dir = 'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/'
#done_file = done_dir + '/*.png'
done_file = glob.glob(done_dir + '*.png')

In [59]:
print(len(done_file))
print(done_file[0])

6446
C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth\[20190701]_{170293173}_ANT.png


In [88]:
done_file[0] = os.path.basename(done_file[0].replace('\\','/'))

In [89]:
done_file[0]

'[20190701]_{170293173}_ANT.png'

In [94]:
group1_Negative

'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth'

In [97]:
##################################
#result 9 = N/A
#result 8 = no final diagonosis
##################################

#변환이 완료된 파일의 목록을 읽어옵니다.
done_dir = 'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done'
done_file = done_dir + '/*.png'  
done_file = glob.glob(done_file.replace('\\','/'))
#정렬합니다.
done_file = natsort.natsorted(done_file)

#파일명의 study date 시작 인덱스입니다. [xxxx.xx.xx]규칙입니다. 0번째는 [이니 1번째 인덱스입니다. 모든 파일이 공통이라 1번째 인덱스로 고정해줍니다.
date_start = 1
#인덱싱에 사용할 count 변수입니다.
count = 1
not_count = 0
error_count = 0 
na_count = 0
final_count = 0

error_list = list() # 일치하는 엑셀 정보 없음
not_list = list() # 그룹조건에 없음
for i in done_file:
    #각 파일에서 규칙을 기준으로 인덱스를 조회합니다.
    #모든 파일이 동일한 규칙을 가지고 있기에 동일한 인덱스지만 중간에 파일명 형식이 다른 경우가 있어 각 파일별로 불러옵니다.
    date_end = os.path.basename(i).find(']')
    id_start = os.path.basename(i).find('{') + 1
    id_end = os.path.basename(i).find('}')
    
    #불러온 인덱스로 study date와 patient id를 불러옵니다.
    file_date = os.path.basename(i)[date_start:date_end]
    #비교를 위해 int(정수형)으로 변환하였습니다.
    file_id = int(os.path.basename(i)[id_start:id_end])
    exel_id = df.loc[count]['PatID']
    
    #읽어온 patient id와 study date를 출력합니다.
    #print(file_id, file_date)
    #비교를 위해 변환하여 비교하고 파일의 patient id와 excel의 patient id와 동일한 인덱스를 찾아 temp에 저장합니다. 
    temp = df[(df['PatID'].astype(int) == int(file_id))]
    #patient id에서 일치하는 경우 study date를 한 번 더 비교하여 일치하는 정보를 찾습니다. 
    match = temp[temp['ExamDate'].astype(int) == int(file_date)]
    if(len(match) == 0):
        error_count = error_count + 1 
        error_list.append(i)
        continue
    #위에서 찾은 정보로 해당 파일의 group을 저장합니다.
    group = match['Group'].astype(int)
    
    #데이터프레임에서 원소만 저장합니다.
    group = group.iloc[0]
    
    #비교를 위해 정수형으로 변환하였습니다.
    result = match['Result'].astype(int)
    #데이터프레임에서 원소만 저장합니다.
    result = result.iloc[0]
    
    #group에 따라 지정할 파일 경로입니다.
    group1_Negative = os.path.dirname(i.replace('done','class/Group1/Negative'))
    group1_Positive = os.path.dirname(i.replace('done','class/Group1/Positive'))
    
    group2_Negative = os.path.dirname(i.replace('done','class/Group2/Negative'))
    group2_Positive = os.path.dirname(i.replace('done','class/Group2/Positive'))
    
    group3_Negative = os.path.dirname(i.replace('done','class/Group3/Negative'))
    group3_Positive = os.path.dirname(i.replace('done','class/Group3/Positive'))
    
    #경로가 없을 경우 생성합니다.
    if not(os.path.isdir(group1_Negative)):
        os.makedirs(os.path.join(group1_Negative))
        os.makedirs(os.path.join(group1_Positive))
        
        os.makedirs(os.path.join(group2_Negative))
        os.makedirs(os.path.join(group2_Positive))
        
        os.makedirs(os.path.join(group3_Negative))
        os.makedirs(os.path.join(group3_Positive))
        
    #설정한 group과 조건에 따라 분류합니다.
    #N/A와 no final diagnosis의 경우 제외합니다.
    if(group == 1):
        if(result == 0):
            #print('group1_음성')
            shutil.copy2(i,group1_Negative)
        elif(result == 1):
            #print('group1_음성')
            shutil.copy2(i,group1_Negative)
        elif(result == 2):
            #print('group1_양성')
            shutil.copy2(i,group1_Positive)
        elif(result == 3):
            #print('group1_양성')
            shutil.copy2(i,group1_Positive)
        elif(result == 9):
            print('group1_N/A')
            na_count = na_count + 1
        elif(result == 8):
            print('group1_no_final_diagnosis')
            final_count = final_count + 1
    elif(group == 2):
        if(result == 0):
            #print('group2_음성')
            shutil.copy2(i,group2_Negative)
        elif(result == 1):
            #print('group2_음성')
            shutil.copy2(i,group2_Negative)
        elif(result == 2):
            #print('group2_양성')
            shutil.copy2(i,group2_Positive)
        elif(result == 3):
            #print('group2_양성')
            shutil.copy2(i,group2_Positive)
        elif(result == 9):
            print('group2_N/A')
            na_count = na_count + 1
        elif(result == 8):
            print('group2_no_final_diagnosis')
            final_count = final_count + 1
    elif(group == 3):
        if(result == 0):
            #print('group3_음성')
            shutil.copy2(i,group3_Negative)
        elif(result == 1):
            #print('group3_양성')
            shutil.copy2(i,group3_Positive)
        elif(result == 2):
            #print('group3_양성')
            shutil.copy2(i,group3_Positive)
        elif(result == 3):
            #print('group3_양성')
            shutil.copy2(i,group3_Positive)
        elif(result == 9):
            print('group3_N/A')
            na_count = na_count + 1
        elif(result == 8):
            print('group3_no_final_diagnosis')
            final_count = final_count + 1
    else:
        print('except!_' + i)
        not_count = not_count + 1
        not_list.append(i)
        continue
        
    count = count + 1
print(not_count)
print(error_count)
print(na_count)
print(final_count)

except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20190708]_{190174363}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20190724]_{100050833}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20190801]_{190220723}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20190807]_{190041333}_ANT.png
group2_no_final_diagnosis
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20190812]_{190224243}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20190822]_{000001743}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20190826]_{000269223}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20190910]_{190268583}_ANT.png
group2_no_final_diagnosis
except!_C:/Users/starlabs/De

except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20210531]_{210181393}_ANT.png
group1_no_final_diagnosis
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20210602]_{210046363}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20210614]_{960197493}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20210626]_{090056263}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20210626]_{160234703}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20210701]_{170278203}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20210703]_{210262643}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\[20210706]_{000170023}_ANT.png
except!_C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_D

In [98]:
print(error_list)
display(not_list)

['C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\\[20220615]_{190211003}_ANT.png']


['C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\\[20190708]_{190174363}_ANT.png',
 'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\\[20190724]_{100050833}_ANT.png',
 'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\\[20190801]_{190220723}_ANT.png',
 'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\\[20190807]_{190041333}_ANT.png',
 'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\\[20190812]_{190224243}_ANT.png',
 'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\\[20190822]_{000001743}_ANT.png',
 'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\\[20190826]_{000269223}_ANT.png',
 'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\\[20190910]_{190268583}_ANT.png',
 'C:/Users/starlabs/Desktop/BoneScan/Bone_Scan_Data/convert_concat_depth/done\\[20190920]_{950313443}_AN

### 이후는 참고용, 지워도됨

In [ ]:
all_list = glob.glob('C:/Users/BrainK/Downloads/Bone_Scan_TEST/ANT/*.png')

In [ ]:
date_start = 1
png_list = list()
for i in all_list:
    date_end = os.path.basename(i).find(']')
    id_start = os.path.basename(i).find('{') + 1
    id_end = os.path.basename(i).find('}')
    
    file_date = os.path.basename(i)[date_start:date_end]
    file_id = os.path.basename(i)[id_start:id_end]
    
    png_list.append(file_date + '_' + file_id)    

In [ ]:
count = 0
for i in range(len(png_list)-1):
    if png_list[i] == png_list[i-1]:
        print(png_list[i])
        count = count + 1
        
print(count)

In [ ]:
class_list = glob.glob('C:/Users/BrainK/Downloads/Bone_Scan_TEST/class/*/*/*.png')
print(len(class_list))